# Crypto Data Download
We will use historcal cryptocurrencies data publicully availble on Binance exchange website (https://www.binance.com/en-GB/landing/data).
API documentation is availble here: https://github.com/binance/binance-public-data

### The top 18 cryptocurrencies in 2024
Based on the market capitalisation as per https://coinmarketcap.com portal.
* BTC - Bitcoin
* ETH - Ethereum
* BNB - Binance Coin
* SOL - Solana
* XRP - Ripple
* ADA - Cardano
* AVAX - Avalanche
* DOGE - Dogecoin
* TRX - TRON
* LINK - Chainlink
* DOT - Polkadot
* MATIC - Polygon
* ICP - Internet Computer
* SHIB - Shiba Inu
* BCH - Bitcoin Cash
* LTC - Litecoin
* ATOM - Cosmos
* ETC - Ethereum Classic

In [1]:
import currencies_lib

currencies = currencies_lib.store_to_file()
currencies

,BTC,ETH,BNB,SOL,XRP,ADA,AVAX,DOGE,TRX,LINK,DOT,MATIC,ICP,SHIB,BCH,LTC,ATOM,ETC
0,Bitcoin,Ethereum,Binance Coin,Solana,Ripple,Cardano,Avalanche,Dogecoin,TRON,Chainlink,Polkadot,Polygon,Internet Computer,Shiba Inu,Bitcoin Cash,Litecoin,Cosmos,Ethereum Classic


### Price data
We will download price data in candlestick format with 1 hour interval or higher, groped into monthly files for all currencies defined above.
Time frame for the price data is from Jan-2024 to Dec-2024.

In [2]:
import binance_data as binance

start_period, end_period = ("2024-01-01", "2024-05-01")
# change interval of tick datapoints here
interval = binance.SUPPORTED_INTERVALS[6] # 6 stands for 1 hour data
print("Interval: ", interval)

data_files = binance.download_data_files(start_period, end_period, currencies, data_type="kline", interval=interval)

Interval:  1h


In [3]:
data_files["BTC"]

[DataFile(currency='BTC', interval='1h', date=datetime.date(2024, 1, 31), filename='data/BTCUSDT-1h-2024-01.zip'),
 DataFile(currency='BTC', interval='1h', date=datetime.date(2024, 2, 29), filename='data/BTCUSDT-1h-2024-02.zip'),
 DataFile(currency='BTC', interval='1h', date=datetime.date(2024, 3, 31), filename='data/BTCUSDT-1h-2024-03.zip'),
 DataFile(currency='BTC', interval='1h', date=datetime.date(2024, 4, 30), filename='data/BTCUSDT-1h-2024-04.zip')]

In [4]:
print(binance.FIELDS_KLINE)

{'open_time': 'Kline Open time in unix time format', 'open': 'Open Price', 'high': 'High Price', 'low': 'Low Price', 'close': 'Close Price', 'volume': 'Volume', 'close_time': 'Kline Close time in unix time format', 'quote_volume': 'Quote Asset Volume', 'count': 'Number of Trades', 'taker_buy_volume': 'Taker buy base asset volume during this period', 'taker_buy_quote_volume': 'Taker buy quote asset volume during this period', 'ignore': 'Ignore'}


In [5]:
import pandas as pd

def read_all_data_files(data_files):
    return [pd.read_csv(data_file.filename, header=None, names=binance.FIELDS_KLINE.keys()) for data_file in data_files]

data_frames = {currency: pd.concat(read_all_data_files(data_files[currency])) for currency in currencies}
data_frames["BTC"]

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
0,1704067200000,42283.58,42554.57,42261.02,42475.23,1271.68108,1704070799999,5.395725e+07,47134,682.57581,2.895742e+07,0
1,1704070800000,42475.23,42775.00,42431.65,42613.56,1196.37856,1704074399999,5.098489e+07,50396,712.32227,3.035565e+07,0
2,1704074400000,42613.57,42638.41,42500.00,42581.10,685.21980,1704077999999,2.916738e+07,29863,288.98864,1.230102e+07,0
3,1704078000000,42581.09,42586.64,42230.08,42330.49,794.80391,1704081599999,3.370905e+07,38620,356.37209,1.511300e+07,0
4,1704081600000,42330.50,42399.99,42209.46,42399.99,715.41760,1704085199999,3.027162e+07,36038,371.12012,1.570362e+07,0
...,...,...,...,...,...,...,...,...,...,...,...,...
715,1714503600000,60403.68,60503.87,59191.60,59292.95,5586.14319,1714507199999,3.342923e+08,168691,2089.40520,1.250536e+08,0
716,1714507200000,59292.95,60177.46,59274.75,59958.62,3414.88045,1714510799999,2.042664e+08,103846,1698.69261,1.016049e+08,0
717,1714510800000,59958.62,60400.00,59952.49,60180.00,1151.18801,1714514399999,6.930477e+07,48063,603.09852,3.630523e+07,0
718,1714514400000,60180.00,61070.71,60156.00,60589.79,1945.24597,1714517999999,1.179662e+08,77093,978.69404,5.935423e+07,0


In [6]:
for data_frame in data_frames.values():
    data_frame["open_time"] = pd.to_datetime(data_frame["open_time"], unit="ms")
    data_frame["close_time"] = pd.to_datetime(data_frame["close_time"], unit="ms")
    data_frame.drop(columns=["ignore"], inplace=True)
    data_frame.set_index("open_time", inplace=True)

data_frames["BTC"]

,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume
open_time,,,,,,,,,,
2024-01-01 00:00:00,42283.58,42554.57,42261.02,42475.23,1271.68108,2024-01-01 00:59:59.999,5.395725e+07,47134,682.57581,2.895742e+07
2024-01-01 01:00:00,42475.23,42775.00,42431.65,42613.56,1196.37856,2024-01-01 01:59:59.999,5.098489e+07,50396,712.32227,3.035565e+07
2024-01-01 02:00:00,42613.57,42638.41,42500.00,42581.10,685.21980,2024-01-01 02:59:59.999,2.916738e+07,29863,288.98864,1.230102e+07
2024-01-01 03:00:00,42581.09,42586.64,42230.08,42330.49,794.80391,2024-01-01 03:59:59.999,3.370905e+07,38620,356.37209,1.511300e+07
2024-01-01 04:00:00,42330.50,42399.99,42209.46,42399.99,715.41760,2024-01-01 04:59:59.999,3.027162e+07,36038,371.12012,1.570362e+07
...,...,...,...,...,...,...,...,...,...,...
2024-04-30 19:00:00,60403.68,60503.87,59191.60,59292.95,5586.14319,2024-04-30 19:59:59.999,3.342923e+08,168691,2089.40520,1.250536e+08
2024-04-30 20:00:00,59292.95,60177.46,59274.75,59958.62,3414.88045,2024-04-30 20:59:59.999,2.042664e+08,103846,1698.69261,1.016049e+08
2024-04-30 21:00:00,59958.62,60400.00,59952.49,60180.00,1151.18801,2024-04-30 21:59:59.999,6.930477e+07,48063,603.09852,3.630523e+07


In [7]:
import hvplot
import hvplot.pandas
from itertools import accumulate

list(accumulate((data_frames[currency].hvplot.line(y="close", xlabel="Date", ylabel=currency) for currency in data_frames), lambda x, y: x*y))[-1]

:Overlay
   .Curve.I     :Curve   [open_time]   (close)
   .Curve.II    :Curve   [open_time]   (close)
   .Curve.III   :Curve   [open_time]   (close)
   .Curve.IV    :Curve   [open_time]   (close)
   .Curve.V     :Curve   [open_time]   (close)
   .Curve.VI    :Curve   [open_time]   (close)
   .Curve.VII   :Curve   [open_time]   (close)
   .Curve.VIII  :Curve   [open_time]   (close)
   .Curve.IX    :Curve   [open_time]   (close)
   .Curve.X     :Curve   [open_time]   (close)
   .Curve.XI    :Curve   [open_time]   (close)
   .Curve.XII   :Curve   [open_time]   (close)
   .Curve.XIII  :Curve   [open_time]   (close)
   .Curve.XIV   :Curve   [open_time]   (close)
   .Curve.XV    :Curve   [open_time]   (close)
   .Curve.XVI   :Curve   [open_time]   (close)
   .Curve.XVII  :Curve   [open_time]   (close)
   .Curve.XVIII :Curve   [open_time]   (close)

In [8]:
common_df = data_frames["BTC"]
columns_to_rename = {column:f"{column}_BTC" for column in common_df.columns}
[common_df := common_df.join(data_frames[curreny], how="outer", rsuffix=f"_{curreny}") for curreny in data_frames if curreny != "BTC"]
common_df = common_df.rename(columns=columns_to_rename)
common_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2904 entries, 2024-01-01 00:00:00 to 2024-04-30 23:00:00
Columns: 180 entries, open_BTC to taker_buy_quote_volume_ETC
dtypes: datetime64[ns](18), float64(144), int64(18)
memory usage: 4.0 MB


In [9]:
prices = common_df[[column for column in common_df.columns if column.startswith("close_") and not column.startswith("close_time")]]
prices.index.rename("Date", inplace=True)
prices = prices.rename(columns={column:column[column.rfind('_') + 1:] for column in prices.columns})
prices.info()
prices.to_csv(f"output/prices-{interval}.csv")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2904 entries, 2024-01-01 00:00:00 to 2024-04-30 23:00:00
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   BTC     2904 non-null   float64
 1   ETH     2904 non-null   float64
 2   BNB     2904 non-null   float64
 3   SOL     2904 non-null   float64
 4   XRP     2904 non-null   float64
 5   ADA     2904 non-null   float64
 6   AVAX    2904 non-null   float64
 7   DOGE    2904 non-null   float64
 8   TRX     2904 non-null   float64
 9   LINK    2904 non-null   float64
 10  DOT     2904 non-null   float64
 11  MATIC   2904 non-null   float64
 12  ICP     2904 non-null   float64
 13  SHIB    2904 non-null   float64
 14  BCH     2904 non-null   float64
 15  LTC     2904 non-null   float64
 16  ATOM    2904 non-null   float64
 17  ETC     2904 non-null   float64
dtypes: float64(18)
memory usage: 431.1 KB


In [10]:
prices_dataset_summary = prices.describe().transpose()
prices_dataset_summary.to_csv("output/step_01_Prices_Dataset_Description.csv")
prices_dataset_summary

,count,mean,std,min,25%,50%,75%,max
BTC,2904.0,56512.632283,11111.445598,38767.740000,43920.427500,61375.505000,66766.750000,73577.350000
ETH,2904.0,2997.317689,534.779628,2183.780000,2474.310000,3071.270000,3434.027500,4067.790000
BNB,2904.0,440.847211,123.576492,288.200000,313.700000,407.800000,575.800000,632.700000
SOL,2904.0,131.631932,34.836095,79.370000,100.310000,127.730000,158.700000,209.090000
XRP,2904.0,0.569096,0.049075,0.431300,0.526700,0.566600,0.611700,0.734000
ADA,2904.0,0.575780,0.086097,0.412900,0.504775,0.574500,0.630225,0.799800
AVAX,2904.0,41.094008,7.654124,27.670000,35.427500,38.810000,46.117500,63.840000
DOGE,2904.0,0.125787,0.045131,0.075760,0.081380,0.127825,0.163727,0.227180
TRX,2904.0,0.120643,0.010951,0.101630,0.111767,0.119360,0.127005,0.143720
LINK,2904.0,17.162960,2.401907,12.142000,14.802250,17.809500,19.201750,21.680000
